In [1]:
import pandas as pd
import numpy as np

from config import *

In [2]:
events_df = pd.read_csv('../data/raw/EventLevelData.csv')
events_df.head(5)

,MatchID,TeamID,TeamName,PlayerID,PlayerName,EventID,match_period,event_timestamp,type_name,play_pattern_name,...,duel_type_name,pass_outcome_name,pass_end_location_x,pass_end_location_y,pass_height_name,pass_assisted_shot_id,shot_aerial_won,expected_goals,EventValue,MinutesPlayed
0,86,0,North Alps United,0,Julian Martins,ae6af116-3a60-4923-89dc-b47f3c011fcc,2,00:36:45.360,Pass,From Throw In,...,NaN,NaN,86.95,27.46,Ground Pass,NaN,NaN,NaN,-0.000558,95.207767
1,84,0,North Alps United,0,Julian Martins,409df137-d15f-4f6e-863b-c753ee430924,1,00:30:26.359,Pass,From Goal Kick,...,NaN,NaN,83.99,74.25,Ground Pass,NaN,NaN,NaN,-0.001029,87.712067
2,85,4,AFC White Forest,3,Samuel Garnier,544a0991-add5-4625-bdcd-49b9ccda6be0,1,00:08:59.470,Pressure,Regular Play,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001199,97.151300
3,86,0,North Alps United,0,Julian Martins,3433bef8-2f66-4a2e-aa5c-9b99efb3a8d5,1,00:23:17.999,Ball Receipt*,From Throw In,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000302,95.207767
4,84,0,North Alps United,0,Julian Martins,284de480-467c-40be-b38d-8d3237e98e6d,1,00:10:17.349,Carry,Regular Play,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.012863,87.712067


In [12]:
matches_df = pd.read_csv('../data/raw/MatchLevelData.csv')
matches_df.head(5)

,MatchID,MatchDateTime,HomeTeamID,AwayTeamID,HomeElo,AwayElo,HomeTeamXG,AwayTeamXG,HomeGoals,AwayGoals
0,158,2024-11-26 20:00:00,2.0,118.0,1586.922101,1452.824546,1.772981,1.012384,1,0
1,48,2024-09-28 14:30:00,27.0,1.0,1125.956205,1332.964825,0.459880,1.659593,0,2
2,143,2024-09-28 16:30:00,2.0,93.0,1566.872248,1500.869545,1.220850,0.200676,1,1
3,55,2024-05-19 15:00:00,35.0,5.0,1456.428244,1364.767279,1.125992,2.267471,2,0
4,104,2024-09-30 19:00:00,5.0,56.0,1368.433022,1302.558780,1.360136,0.463532,3,1


In [3]:
events_df.columns

Index(['MatchID', 'TeamID', 'TeamName', 'PlayerID', 'PlayerName', 'EventID',
       'match_period', 'event_timestamp', 'type_name', 'play_pattern_name',
       'location_x', 'location_y', 'under_pressure', 'goal', 'free_kick',
       'penalty', 'ball_receipt_outcome_name', 'clearance_aerial_won',
       'carry_end_location_x', 'carry_end_location_y', 'dribble_outcome_name',
       'duel_outcome_name', 'duel_type_name', 'pass_outcome_name',
       'pass_end_location_x', 'pass_end_location_y', 'pass_height_name',
       'pass_assisted_shot_id', 'shot_aerial_won', 'expected_goals',
       'EventValue', 'MinutesPlayed'],
      dtype='object')

In [4]:
print(f"Number of matches in events data: {events_df['MatchID'].nunique()}")
print(f"Number of players in events data: {events_df['PlayerID'].nunique()}")
print(f"Number of event types in events data: {events_df['type_name'].unique()}")

Number of matches in events data: 178
Number of players in events data: 5
Number of event types in events data: ['Pass' 'Pressure' 'Ball Receipt*' 'Carry' 'Duel' 'Dribble' 'Shot'
 'Clearance']


In [14]:
print(f"Number of matches in matches data: {events_df['MatchID'].nunique()}")

Number of matches in matches data: 178


In [5]:
events_df.groupby('PlayerName')['MatchID'].nunique()

PlayerName
Arthur Senyo      35
Benjamin Brody    35
Jordan Bamba      41
Julian Martins    39
Samuel Garnier    32
Name: MatchID, dtype: int64

In [8]:
events_df[EVENT_TIMESTAMP].str.len().unique()

array([12])

In [11]:
pd.to_datetime(events_df[EVENT_TIMESTAMP])

/var/folders/bk/p0hf4p4s2r94_pdnrzmls1yr0000gp/T/ipykernel_42153/68817752.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(events_df[EVENT_TIMESTAMP])


0       2024-12-14 00:36:45.360
1       2024-12-14 00:30:26.359
2       2024-12-14 00:08:59.470
3       2024-12-14 00:23:17.999
4       2024-12-14 00:10:17.349
                  ...          
20546   2024-12-14 00:35:05.240
20547   2024-12-14 00:43:43.177
20548   2024-12-14 00:31:03.711
20549   2024-12-14 00:34:37.431
20550   2024-12-14 00:49:32.651
Name: event_timestamp, Length: 20551, dtype: datetime64[ns]

In [17]:
joined_df = events_df.merge(matches_df, on='MatchID', how='left')
joined_df[joined_df['HomeTeamID'].isnull()][MATCH_ID].unique()

array([147, 127])

In [24]:
test = events_df.groupby(MATCH_ID, as_index=False)['TeamID'].unique()

In [27]:
test2 = matches_df.merge(test, on=MATCH_ID, how='left')

In [30]:
test2['match'] = test2.apply(lambda row: ((row['HomeTeamID'] in (row['TeamID'])) | (row['AwayTeamID'] in (row['TeamID']))), axis=1)

In [33]:
test2[test2['match'] == True]['MatchID'].nunique()

40

In [34]:
print(f"Min x dimension: {events_df[LOCATION_X].min()}")
print(f"Max x dimension: {events_df[LOCATION_X].max()}")
print(f"Min y dimension: {events_df[LOCATION_Y].min()}")
print(f"Max y dimension: {events_df[LOCATION_Y].max()}")

Min x dimension: 0.72
Max x dimension: 120.1
Min y dimension: 0.1
Max y dimension: 80.1
